In [2]:
import pandas as pd
import datetime
from datetime import datetime
from dateutil.parser import parse

In [ ]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [ ]:
df = pd.read_csv("delivery_trip_truck_data.csv")
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df=df.fillna(method='ffill')
df.isna().sum()

In [ ]:
df=df.fillna(method='bfill')
df.isna().sum()

In [ ]:
df['BookingID_Date'] = pd.to_datetime(df['BookingID_Date'])

In [ ]:
df.sample(5)

In [ ]:
df.dtypes

In [ ]:
df['Data_Ping_time'] = pd.to_datetime(df['Data_Ping_time'], format='mixed')

In [ ]:
df['Planned_ETA'] = pd.to_datetime(df['Planned_ETA'], format='mixed')

In [ ]:
df.dtypes

In [ ]:
df['actual_eta'] =  pd.to_datetime(df['actual_eta'], format='mixed')

In [ ]:
df['trip_start_date'] = pd.to_datetime(df['trip_start_date'], format='mixed')

In [ ]:
df['trip_end_date'] = pd.to_datetime(df['trip_end_date'], format='mixed')

In [ ]:
df.sample(5)

In [ ]:
# df.to_csv('truck_data.csv',index=False)

In [ ]:
df['MarketRegular'].value_counts()

In [ ]:
import datetime
import threading
from decimal import *
from time import sleep
from uuid import uuid4, UUID

from confluent_kafka import SerializingProducer
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.avro import AvroSerializer
from confluent_kafka.serialization import StringSerializer

In [ ]:
def delivery_report(err, msg):
    """
    Reports the failure or success of a message delivery.

    Args:
        err (KafkaError): The error that occurred on None on success.

        msg (Message): The message that was produced or failed.

    Note:
        In the delivery report callback the Message.key() and Message.value()
        will be the binary format as encoded by any configured Serializers and
        not the same object that was passed to produce().
        If you wish to pass the original object(s) for key and value to delivery
        report callback we recommend a bound callback or lambda where you pass
        the objects along.

    """
    if err is not None:
        print("Delivery failed for User record {}: {}".format(msg.key(), err))
        return
    print('User record {} successfully produced to {} [{}] at offset {}'.format(
        msg.key(), msg.topic(), msg.partition(), msg.offset()))

# Define Kafka configuration
kafka_config = {
    'bootstrap.servers': 'pkc-60py3.europe-west9.gcp.confluent.cloud:9092',
    'sasl.mechanisms': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': '',
    'sasl.password': ''
}

# Create a Schema Registry client
schema_registry_client = SchemaRegistryClient({
    'url': 'https://psrc-l6o18.us-east-2.aws.confluent.cloud',
    'basic.auth.user.info': '{}:{}'.format('',
     '')
})

# Fetch the latest Avro schema for the value
subject_name = 'delivery_truck_logistics-value'
schema_str = schema_registry_client.get_latest_version(subject_name).schema.schema_str

# Create Avro Serializer for the value
# key_serializer = AvroSerializer(schema_registry_client=schema_registry_client, schema_str='{"type": "string"}')
key_serializer = StringSerializer('utf_8')
avro_serializer = AvroSerializer(schema_registry_client, schema_str)

# Define the SerializingProducer
producer = SerializingProducer({
    'bootstrap.servers': kafka_config['bootstrap.servers'],
    'security.protocol': kafka_config['security.protocol'],
    'sasl.mechanisms': kafka_config['sasl.mechanisms'],
    'sasl.username': kafka_config['sasl.username'],
    'sasl.password': kafka_config['sasl.password'],
    'key.serializer': key_serializer,  # Key will be serialized as a string
    'value.serializer': avro_serializer  # Value will be serialized as Avro
})

# Iterate over DataFrame rows and produce to Kafka
for index, row in df.iterrows():
    # Create a dictionary from the row values
    value = row.to_dict()
    # Produce to Kafka
    producer.produce(topic='delivery_truck_logistics', key=str(index), value=value, on_delivery=delivery_report)
    producer.flush()
    #break

print("All Data successfully published to Kafka")
